## ViraLens Agent POC

This notebook is a proof of concept for the ViraLens Agent. The agent is supposed to do the following:

- Search the web for the latest news on AI using the Tavily API
- Then it will select the news that has the most potential to create the most viral tiktok
- After this, it will do some more research about the news using tavily again. 
- Once it is done with the above, it will generate a script with a hook and everything such that it can be used to create a tiktok video. 

In [26]:
import os

from dotenv import load_dotenv
from tavily import TavilyClient
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field

load_dotenv()

True

In [27]:
def web_search(query: str, max_results: int = 3) -> list[dict]:
    """
    Perform a web search using the Tavily API.

    Args:
        query: The search query string
        max_results: Maximum number of results to return (default: 5)

    Returns:
        dict: Search results from Tavily API containing:
            - results: List of search result objects with title, url, content, score
            - query: The original search query
            - response_time: API response time
    """
    tavily_api_key = os.getenv("TAVILY_API_KEY")

    if not tavily_api_key:
        raise ValueError("TAVILY_API_KEY not found in environment variables")

    client = TavilyClient(api_key=tavily_api_key)

    response = client.search(
        query=query,
        include_answer="basic",
        topic="news",
        search_depth="basic",
        max_results=3,
    )

    return response

In [28]:
results = web_search("What is the latest news on AI?")
results

{'query': 'What is the latest news on AI?',
 'follow_up_questions': None,
 'answer': "The latest news on AI includes Equinox's New Year’s campaign contrasting AI fakery with real human strength, predictions from 100 ad leaders on how AI will reshape marketing in 2026, and Samsung integrating AI into everyday appliances. Additionally, AI-driven scientific breakthroughs in 2025 accelerated progress in robotics, healthcare, and climate modeling, with the federal government investing $3.3 billion in non-defense AI research and development.",
 'images': [],
 'results': [{'url': 'https://adage.com/creativity/work/aa-equinox-question-everything-but-yourself/',
   'title': 'Equinox contrasts AI fakery with real human strength in New Year’s campaign - Ad Age',
   'score': 0.6533265,
   'published_date': 'Mon, 05 Jan 2026 17:45:00 GMT',
   'content': '# Equinox contrasts AI fakery with real human strength in New Year’s campaign. January 05, 2026 12:45 PM EST. It didn’t take long for the AI bashi

In [29]:
news_articles = [
    {"title": article.get("title", ""), "content": article.get("content", "")}
    for article in results.get("results", [])
]

news_articles

[{'title': 'Equinox contrasts AI fakery with real human strength in New Year’s campaign - Ad Age',
  'content': '# Equinox contrasts AI fakery with real human strength in New Year’s campaign. January 05, 2026 12:45 PM EST. It didn’t take long for the AI bashing to start up again in 2026. ## Featured Stories. 100 ad leaders predict 2026 marketing trends—what’s next for AI, agencies, creativity, media and more. Explore how 100 ad leaders say AI, data, agencies, CTV, creators and brand strategy will reshape marketing in 2026. Samsung is putting AI into everyday appliances, from fridges that track what you eat to vacuums that can police pet activity. Ad Age’s 2026 Business Forecast—how you can prepare for the year. Winter Olympics 2026—everything brands need to know. ## Latest News. 5 biggest ad tech moments—and what we can learn from them. Tree Hut brings its stretchy social energy to first major TV campaign. Rinse’s charming ads say greatness starts with not doing laundry. Secret Level’s

In [30]:
article_chooser_system_prompt = """
You are a news article chooser. You are given a list of news articles and you need to choose the one that has the most potential to create the most viral tiktok.
"""

article_chooser_user_prompt = """
Here are the news articles:
{news_articles}
"""

In [31]:
class ViralArticle(BaseModel):
    title: str = Field(..., description="The title of the chosen article")
    content: str = Field(..., description="The full content of the chosen article")
    reason: str = Field(..., description="Explanation of why this article has the most viral potential")

In [ ]:
model = ChatGoogleGenerativeAI(model="gemini-3-flash-preview", api_key=os.getenv("GEMINI_API_KEY"))
model_with_structure = model.with_structured_output(ViralArticle)

formatted_user_prompt = article_chooser_user_prompt.format(news_articles=news_articles)

messages = [
    {"role": "system", "content": article_chooser_system_prompt},
    {"role": "user", "content": formatted_user_prompt}
]

response = model_with_structure.invoke(messages)
print(response.model_dump_json(indent=2))

{"title":"Equinox contrasts AI fakery with real human strength in New Year’s campaign - Ad Age","content":"# Equinox contrasts AI fakery with real human strength in New Year’s campaign. January 05, 2026 12:45 PM EST. It didn’t take long for the AI bashing to start up again in 2026. ## Featured Stories. 100 ad leaders predict 2026 marketing trends—what’s next for AI, agencies, creativity, media and more. Explore how 100 ad leaders say AI, data, agencies, CTV, creators and brand strategy will reshape marketing in 2026. Samsung is putting AI into everyday appliances, from fridges that track what you eat to vacuums that can police pet activity. Ad Age’s 2026 Business Forecast—how you can prepare for the year. Winter Olympics 2026—everything brands need to know. ## Latest News. 5 biggest ad tech moments—and what we can learn from them. Tree Hut brings its stretchy social energy to first major TV campaign. Rinse’s charming ads say greatness starts with not doing laundry. Secret Level’s two h

In [33]:
class TikTokScript(BaseModel):
    hook: str = Field(..., description="The opening hook (first 3-5 seconds) that grabs attention immediately")
    main_content: str = Field(..., description="The main story/content for the middle section (15-20 seconds)")
    call_to_action: str = Field(..., description="The closing call-to-action to encourage engagement (last 3-5 seconds)")
    visual_notes: str = Field(..., description="Suggestions for visual elements, transitions, or on-screen text")

In [34]:
script_generator_system_prompt = """
You are a viral TikTok script writer. You create engaging, attention-grabbing scripts for 30-second videos that maximize views, likes, and shares.

Your scripts must:
- Start with a strong hook in the first 3-5 seconds
- Tell a compelling story in 15-20 seconds
- End with a clear call-to-action in the last 3-5 seconds
- Be optimized for TikTok's algorithm (trending sounds, hashtags, engagement triggers)
- Use conversational, relatable language
- Include visual suggestions that enhance the narrative
"""

script_generator_user_prompt = """
Based on this news article, create a viral TikTok script for a 30-second video:

Title: {title}
Content: {content}
Reason for viral potential: {reason}

Generate a script that will maximize engagement and go viral.
"""

In [35]:
script_model = model.with_structured_output(TikTokScript)

formatted_script_prompt = script_generator_user_prompt.format(
    title=response.title,
    content=response.content,
    reason=response.reason
)

script_messages = [
    {"role": "system", "content": script_generator_system_prompt},
    {"role": "user", "content": formatted_script_prompt}
]

script_response = script_model.invoke(script_messages)
print(script_response.model_dump_json(indent=2))

{
  "hook": "Stop scrolling if you are tired of seeing fake AI perfection everywhere.",
  "main_content": "Equinox just dropped their 2026 New Year campaign and it is basically a massive middle finger to the AI hype. While companies like Samsung are putting AI in your fridge and your vacuum, Equinox is calling out the fakery. They are contrasting uncanny AI-generated images with raw, gritty human strength. Think about it: you can prompt a computer to write code, but you can not prompt your way into a six-pack. In a world of deepfakes, the only thing left that is actually real is your sweat and your effort. This is the ultimate Human versus Machine showdown.",
  "call_to_action": "Are you Team Human or Team AI? Let us argue in the comments below.",
  "visual_notes": "Start with a glitchy AI filter over the creator face that snaps to a raw high-definition gym shot. Use fast cuts between 'perfect' AI-generated fitness models and real people sweating and struggling with weights. Add bold o